In [4]:
from datetime import datetime
import pandas as pd
import numpy as np
from dateutil.parser import parse
pd.set_option("display.max_rows", 15)
pd.set_option("display.max_columns", 30)
import datetime
from dateutil.parser import parse
import math

from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import ipynbname
nb_fname = ipynbname.name()

### Research Context and Goal
The goal of the modelling at hand is to predict toxicity test values for the toxin type DSP in seashell based on the abundance of micro-algae in combination with available environmental data. Marine biologists are also interested to understand which species of micro-algae affects the accumulation of toxins in the seashells. 

The monitoring of abundance of micro-algae is carried out by the National Institute of Biology, Marine Biological Station Piran. The toxicity tests are performed by the National Veterinary Institute, Faculty of Veterinary Medicine (UL), which depending on the results of these tests administers bans on sales of shellfish. 

# Data preprocessing

## 1. Micro-algae dataset

In [5]:
# Create micro-algae dataframe (empty cells -> NaN)
df_alg = pd.read_csv("data/archive/Algae_Podatki_1994_onwards_24062021_Martin_1_0.csv", na_values = ["NA", "?"], sep=";")

# Keep only columns: PSP,  DSP,  Dinophysis species of special interest,  DSP_like,  ASP
df_alg = df_alg[["date", "sampling station", "sampling depth", "sampling method","PSP", "DSP", 
                 "DSP_like", "ASP", "Dinophysis caudata", "Dinophysis fortii", "Phalacroma rotundatum", 
                 "Dinophysis sacculus", "Dinophysis tripos"]]
#replace , with .
df_alg.replace(",", ".", regex=True, inplace=True)

# create timestamp (Option to use parameter dayfirst=True, but seems unreliable)
df_alg["date"] = pd.to_datetime(df_alg["date"])

# sort by date attribute
df_alg = df_alg.sort_values('date')

# change str numbers to floats
df_alg.loc[:,"PSP":"Dinophysis tripos"] = df_alg.loc[:,"PSP":"Dinophysis tripos"].astype(float)

df_alg

FileNotFoundError: [Errno 2] No such file or directory: 'data/archive/Algae_Podatki_1994_onwards_24062021_Martin_1_0.csv'

In [5]:
# Rescale values of samples with sampling method type "integrated phytoplankton net" by factor
phyto_net_factor = 100
idxs_net = df_alg[df_alg["sampling method"] == "phytoplankton net"].index
df_alg.loc[idxs_net, "PSP":"Dinophysis tripos"] = df_alg.loc[idxs_net, "PSP":"Dinophysis tripos"]*phyto_net_factor

In [6]:
# Resolve multiple samples from the same day and location with different sampling depth by keeping only 
# those with the highest abundace value of priority microalgae type (DSP).
df_alg_depth = df_alg.copy()
df_alg_depth["org_index"] = df_alg_depth.index
df_max_DSP = df_alg_depth.groupby(["date", "sampling station"], 
                                  as_index=False).agg({"DSP": lambda x: np.argmax(x), "org_index": lambda x: tuple(x)})
# df_max_DSP

# Use max value index of tuple consisting of original indexes to find the maxid (for DSP) of original dataset
maxid_list = []
for df_idx, group_idxs in df_max_DSP.org_index.iteritems():
    tuple_idx = df_max_DSP.loc[df_idx, "DSP"]
    maxid_list.append(group_idxs[tuple_idx]) 

df_alg = df_alg.iloc[maxid_list]

In [7]:
# Count of missing values
df_alg.isnull().sum()

date                       0
sampling station           0
sampling depth             0
sampling method            0
PSP                        5
DSP                        1
DSP_like                  60
ASP                      344
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
dtype: int64

In [6]:
# Create pickle
df_alg.to_pickle("objects/df_alg")

## 2. Toxins dataset

In [10]:
# Create toxins dataframe
# empty cells [test not available, not in the program of testing] -> NaN
df_tox = pd.read_csv("data/archive/Toxins_Podatki_1994_onwards_24062021_Martin_1_0.csv", 
                     na_values=["x", "? (no data)", "unreliable", 
                                "niso več delali analiz, ker nimajo školjk konzumne velikosti"], sep=";")  
df_tox = df_tox.drop(columns=["data source", "PSP toxins", "ASP toxins", "yessotoxins"])

# # create timestamp
df_tox["date"] = pd.to_datetime(df_tox["date"], infer_datetime_format=True)

# # sort by date attribute
df_tox = df_tox.sort_values('date')

# # set DateTime as index
df_tox.set_index('date', inplace=True)

df_tox

,sampling station,lipophylic toxins,lipophylic toxins (OA (µg/kg)),lipophylic toxins (DTX2 (µg/kg)),ban start,ban stop
date,,,,,,
1989-09-15,35,poz,NaN,NaN,NaN,NaN
1989-09-15,24,poz,NaN,NaN,NaN,NaN
1989-09-25,35,NaN,NaN,NaN,y,NaN
1989-09-25,24,NaN,NaN,NaN,y,NaN
1989-09-25,0DB2,poz,NaN,NaN,y,NaN
...,...,...,...,...,...,...
2019-12-20,35,< MD,< MD,< MD,NaN,NaN
2019-12-20,0DB2,< MD,< MD,< MD,NaN,NaN
2019-12-30,24,< MD,< MD,< MD,NaN,NaN


In [11]:
# Adjustments to the dataset values ["< MD", "< 3 mg DA/kg", "< 0,2 mg DA/kg", 
# "0,2 mg DA/kg (neg)", "0.29 mg/kg", "neg (na meji)"] -> neg
df_tox.replace(to_replace=["< MD", "< 3 mg DA/kg", "< 0,2 mg DA/kg", "0,2 mg DA/kg (neg)", 
                           "0.29 mg/kg", "neg (na meji)"], value="neg", inplace=True)
# ">320, ocena 920" -> 920 [estimated value], 
df_tox.replace(">320, ocena 920", 920, inplace=True)
#>320, ocena 1580" -> 1580
df_tox.replace(">320, ocena 1580", 1580, inplace=True)
#">320, ocena 1880" -> 1880
df_tox.replace(">320, ocena 1880", 1880, inplace=True)
#">320, ocena 336" -> 336
df_tox.replace(">320, ocena 336", 336, inplace=True)
# ">320, ocena 470" -> 470, 
df_tox.replace(">320, ocena 470", 470, inplace=True)
#">240, ocena 510" -> 510, 
df_tox.replace(">240, ocena 510", 510, inplace=True)

# at index 947: yessotoxins="0.29 mg/kg" [need to convert to poz / neg); 
# Or should we ignore yesstoxins since only one 4 values?

In [12]:
# Standardise station names
df_tox["sampling station"].replace("0035", 35, inplace=True)
df_tox["sampling station"].replace("0024", 24, inplace=True)

# Change station names to str
df_tox["sampling station"] = df_tox["sampling station"].astype(str)

In [13]:
# Mapping num values -> "poz" or "neg" based on treshold (173 µg/kg = legal limit concentration for DSP toxins, 
# that induce shellfish harvesting ban)
numbers = [str(x) for x in range(10)]
for idx, value in df_tox["lipophylic toxins (OA (µg/kg))"].iteritems():
    if any(x in str(value) for x in numbers):
        if int(value) >= 173:
            #print(f"old value: {value}")
            value = "poz"
            #print(f"new value: {value}")
            df_tox["lipophylic toxins (OA (µg/kg))"][idx] = value
        else:
            #print(f"old value: {value}")
            value = "neg"
            #print(f"new value: {value}")
            df_tox["lipophylic toxins (OA (µg/kg))"][idx] = value

In [14]:
# Joining lipophylic toxins into one column.
# Just copying values from "lipophylic toxins (OA (µg/kg))"[930:] to "lipophylic toxins" and 
# keep the latter as "lipophylic toxins (DTX2 (µg/kg))" has no positive values.
df_tox["lipophylic toxins"][930:] = df_tox["lipophylic toxins (OA (µg/kg))"][930:]

# Remove columns "lipophylic toxins (OA (µg/kg))" and "lipophylic toxins (DTX2 (µg/kg))" from dataframe
df_tox.drop(columns=["lipophylic toxins (OA (µg/kg))", "lipophylic toxins (DTX2 (µg/kg))"], inplace=True)
print(f"lipophylic toxins classes:") 
df_tox["lipophylic toxins"].value_counts(dropna=False)



lipophylic toxins classes:


neg    1066
NaN     159
poz     148
Name: lipophylic toxins, dtype: int64

In [15]:
# Print out toxins dataset
pd.set_option('display.max_rows', 15)
df_tox

,sampling station,lipophylic toxins,ban start,ban stop
date,,,,
1989-09-15,35,poz,NaN,NaN
1989-09-15,24,poz,NaN,NaN
1989-09-25,35,NaN,y,NaN
1989-09-25,24,NaN,y,NaN
1989-09-25,0DB2,poz,y,NaN
...,...,...,...,...
2019-12-20,35,neg,NaN,NaN
2019-12-20,0DB2,neg,NaN,NaN
2019-12-30,24,neg,NaN,NaN


In [16]:
# Descriptive statistics of toxins dataset
df_tox.describe()

,sampling station,lipophylic toxins,ban start,ban stop
count,1373,1214,83,83
unique,3,2,1,1
top,24,neg,y,y
freq,515,1066,83,83


# 3. Consolidating datasets 

In order to learn a model that can make predictions about the toxicity tests' results from the algae abundance we need to alocate to the algae abundace instances a matching toxicity test, while the datasets have different temporal resolution and frequency of sampling. We will do this separately for each of the three locations.

Divide algae dataset by locations 35, 24 and 0DB2.

In [17]:
# df_alg for station 35
df_alg_35 = df_alg[df_alg["sampling station"] == "35"].copy()
# df for station 24
df_alg_24 = df_alg[df_alg["sampling station"] == "24"].copy()
# df for station 0DB2
df_alg_0DB2 = df_alg[df_alg["sampling station"] == "0DB2"].copy()

Divide toxin dataset by locations 35, 24 and 0DB2.

In [18]:
# df_tox for station 35
df_tox_35 = df_tox[df_tox["sampling station"] == "35"].copy()
# df_tox for station 24
df_tox_24 = df_tox[df_tox["sampling station"] == "24"].copy()
# df_tox for station 0DB2
df_tox_0DB2 = df_tox[df_tox["sampling station"] == "0DB2"].copy()

Prepare new dataframes for consolidated datasets by locations 35, 24 and 0DB2.

In [23]:
# df_cons for station 35
df_cons_35 = df_alg_35.copy()
df_cons_35["lipophylic_toxins"] = np.nan

# df_cons for station 24
df_cons_24 = df_alg_24.copy()
df_cons_24["lipophylic_toxins"] = np.nan

# df_cons for station 0DB2 
df_cons_0DB2 = df_alg_0DB2.copy()
df_cons_0DB2["lipophylic_toxins"] = np.nan

### 3.1 Consolidation of abundance and toxin data

In [24]:
pd.set_option("display.max_rows", 15)
df_tox

,sampling station,lipophylic toxins,ban start,ban stop
date,,,,
1989-09-15,35,poz,NaN,NaN
1989-09-15,24,poz,NaN,NaN
1989-09-25,35,NaN,y,NaN
1989-09-25,24,NaN,y,NaN
1989-09-25,0DB2,poz,y,NaN
...,...,...,...,...
2019-12-20,35,neg,NaN,NaN
2019-12-20,0DB2,neg,NaN,NaN
2019-12-30,24,neg,NaN,NaN


In [25]:
# Loop to add appropriate toxin test results to algae samples; add first toxin test 
# after timestamp of an algae sample but no older tests then 30 days.
max_range = pd.Timedelta(30, unit="day")
min_range = pd.Timedelta(0, unit="day")

df_cons_list = [df_cons_35, df_cons_24, df_cons_0DB2]
df_tox_list = [df_tox_35, df_tox_24, df_tox_0DB2]


for _ in range(3):           
    matches = 0
    # Iterate over consolidated df and select a date of the algae sample
    for id_alg, date_alg in df_cons_list[_]["date"].iteritems():
        # Iterate over toxin df and select the date of test 
        tox_list = []
        for date_tox in df_tox_list[_]["lipophylic toxins"].index:
            # Calculate the timespan between algae sample and toxin test 
            timespan = date_tox - date_alg
            # If timespan within 30 days
            if min_range <= timespan <= max_range:
                # The toxin result on first acceptable toxin test day (date_tox) 
                tox_result = df_tox_list[_].loc[date_tox, "lipophylic toxins"]
                # Use only non-NaN tox_result (neg/pos)
                if type(tox_result) == pd.Series:
                    print("Warning: duplicated dates", tox_result)
                    continue
                if tox_result != np.NaN:
                    tox_list.append((tox_result, timespan))
                    # We list matching results but take the closest one.
        tox_list.sort(key= lambda x: x[1])
        if len(tox_list) > 0:
            # print(tox_list)
            df_cons_list[_].loc[id_alg, "lipophylic_toxins"] = tox_list[0][0]
            matches += 1
            
    print(f"Found matching test dates for {_}: {matches}")

Found matching test dates for 0: 402
2004-10-05    neg
2004-10-05    poz
Name: lipophylic toxins, dtype: object
2004-10-05    neg
2004-10-05    poz
Name: lipophylic toxins, dtype: object
2004-10-05    neg
2004-10-05    poz
Name: lipophylic toxins, dtype: object
2004-10-05    neg
2004-10-05    poz
Name: lipophylic toxins, dtype: object
2010-08-17    neg
2010-08-17    poz
Name: lipophylic toxins, dtype: object
2010-08-17    neg
2010-08-17    poz
Name: lipophylic toxins, dtype: object
2010-08-17    neg
2010-08-17    poz
Name: lipophylic toxins, dtype: object
2010-08-17    neg
2010-08-17    poz
Name: lipophylic toxins, dtype: object
Found matching test dates for 1: 398
Found matching test dates for 2: 293


#### Consolidate datasets from each location

In [26]:
# Concatenated dataframe from each location to get final consolidated dataset
df_cons = pd.concat([df_cons_35, df_cons_24, df_cons_0DB2])

# sort by date attribute
df_cons = df_cons.sort_values('date')

df_cons

,date,sampling station,sampling depth,sampling method,PSP,DSP,DSP_like,ASP,Dinophysis caudata,Dinophysis fortii,Phalacroma rotundatum,Dinophysis sacculus,Dinophysis tripos,lipophylic_toxins
0,1994-05-17,35,0,Niskin,1206.0,68.0,NaN,NaN,27.0,0.0,21.0,0.0,0.0,NaN
3,1994-05-17,24,0,Niskin,4188.0,17.0,NaN,NaN,8.0,0.0,0.0,0.0,0.0,NaN
6,1994-06-06,0DB2,0,bucket,0.0,27.0,NaN,NaN,16.0,3.0,5.0,0.0,0.0,NaN
7,1994-06-13,24,0,bucket,324.0,23.0,NaN,NaN,3.0,0.0,0.0,0.0,0.0,NaN
8,1994-06-21,35,0,bucket,0.0,20.0,NaN,NaN,8.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578,2020-11-27,35,integrated,hose sampler,0.0,0.0,0.0,10800.0,0.0,0.0,0.0,0.0,0.0,NaN
1579,2020-11-27,0DB2,integrated,hose sampler,10.0,10.0,0.0,5000.0,0.0,0.0,10.0,0.0,0.0,NaN
1581,2020-12-15,35,integrated,hose sampler,0.0,20.0,0.0,1800.0,0.0,0.0,0.0,0.0,0.0,NaN
1580,2020-12-15,24,integrated,hose sampler,10.0,20.0,0.0,1200.0,0.0,0.0,10.0,0.0,0.0,NaN


In [27]:
# Check for unique values and missing values in the final consolidated dataset.
df_cons.lipophylic_toxins.value_counts(dropna=False)

neg    885
NaN    380
poz    118
Name: lipophylic_toxins, dtype: int64

In [21]:
df_cons.isnull().sum()

date                       0
sampling station           0
sampling depth             0
sampling method            0
PSP                        5
DSP                        1
DSP_like                  60
ASP                      344
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
lipophylic_toxins        356
dtype: int64

## Prepare exports for modelling

In [22]:
# Change sampling station names
df_cons["sampling station"].replace("35", "Seca", inplace=True)
df_cons["sampling station"].replace("24", "Strunjan", inplace=True)
df_cons["sampling station"].replace("0DB2", "Debeli_rtic", inplace=True)

In [23]:
# Export csv
df_cons.to_csv("data/HAB_bio-tox_0.csv", sep=",", na_rep="?", index=False)

In [24]:
# Create pickle
df_cons.to_pickle("objects/df_cons")

# Adding environmental and chemical data

## Sun radiation dataset

In [25]:
# Global sun radiation in kWh/m2
df_sun_kWh = pd.read_excel("data/ORIGINAL/okoljski/Soncno sevanje 2000-2016.xlsx", header=3)

## 3.2 ARSO dataset

In [26]:
df_arso = pd.read_csv("data/ORIGINAL/ARSO/ARSO 1993-2022.txt") 
df_arso = df_arso.loc[:, " valid":].copy()
df_arso.rename(columns = {" valid":"date", "trajanje sonca [h]":"sun [h]"}, inplace = True)
df_arso["date"] = pd.to_datetime(df_arso["date"])
df_arso = df_arso.set_index("date")
df_arso = df_arso.sort_values('date')
df_arso

,povp. dnevna T [°C],povp. veter [m/s],povp. rel. vla. [%],povp. tlak [hPa],količina padavin [mm],sun [h]
date,,,,,,
1993-01-01,0.7,6.3,43,1026,0.0,6.6
1993-01-02,-3.4,6.6,46,1027,0.0,4.0
1993-01-03,-1.7,6.5,47,1025,0.0,1.6
1993-01-04,-1.0,3.6,46,1031,0.0,6.4
1993-01-05,-1.4,2.8,59,1036,0.0,7.2
...,...,...,...,...,...,...
2022-06-26,25.3,2.7,57,1015,0.0,14.3
2022-06-27,26.8,2.1,54,1014,0.0,13.6
2022-06-28,27.4,3.2,53,1013,0.0,8.0


### Adding ARSO data to dataset (sun radiation, ...)

In [27]:
# Loop to add appropriate summed sun radiation period before timestamp of an algae sample in the consolidated dataset. (Add precipiation, air temp)
df_cons2 = df_cons.copy()
df_cons2[["sun [h]", "air temp", "wind strength", "precipitation"]] = np.nan

max_range = pd.Timedelta(20, unit="day")
min_range = pd.Timedelta(0, unit="day")

# Iterate over consolidated df and select a date of the algae sample
for id_alg, date_alg in df_cons["date"].iteritems():
    # Iterate over ARSO df and selecet and instances in period and sum radiation values
    sun_values = []
    air_temp_val = []
    wind_str_val = []
    precip_val = []
    for date_arso in df_arso.index:
        # ***dodaj pandas select date range namesto loopa
        # Calculate the timespan between algae sample and sun radiation
        timespan = date_alg - date_arso
        # If timespan within radiation period
        if min_range <= timespan <= max_range:
            # Append value to sun_values list
            sun_values.append(df_arso.loc[date_arso, "sun [h]"])
            air_temp_val.append(df_arso.loc[date_arso, "povp. dnevna T [°C]"])
            wind_str_val.append(df_arso.loc[date_arso, "povp. veter [m/s]"])
            precip_val.append(df_arso.loc[date_arso, "količina padavin [mm]"])
        # Add sum of rad values to df_cons once all period values are found
        
    
    df_cons2.loc[id_alg, "sun [h]"] = sum(sun_values)
    df_cons2.loc[id_alg, "air temp"] = np.mean(air_temp_val)
    df_cons2.loc[id_alg, "wind strength"] = np.mean(wind_str_val)
    df_cons2.loc[id_alg, "precipitation"] = sum(precip_val)

In [28]:
pd.set_option("display.max_rows", 30)
df_cons2.isnull().sum()

date                       0
sampling station           0
sampling depth             0
sampling method            0
PSP                        5
DSP                        1
DSP_like                  60
ASP                      344
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
lipophylic_toxins        356
sun [h]                    0
air temp                   0
wind strength              0
precipitation              0
dtype: int64

## 3.3 Chemical dataset

In [29]:
pd.set_option("display.max_rows", 30)

In [30]:
df_sea = pd.read_excel("data/ORIGINAL2/Fi_Ke_1994-2021_checked.xlsx")
# For start choose 0 depth (but optimisation possible by keeping both depths as separate features!)
# df_sea = df_sea[(df_sea["depth"] == 0)]
df_sea.drop(["depth",  "Org-N", "TOT-N", "TOT-P", "O2_sat"], axis=1, inplace=True)

df_sea["datetime"] = pd.to_datetime(df_sea["datetime"])
# df_sea = df_sea.set_index("datetime")
df_sea = df_sea.sort_values('datetime')

# Change sampling station names and combine close sampling stations
df_sea["station_id"].replace(["0035", "00MA"], "Seca", inplace=True)
df_sea["station_id"].replace(["0024", "000F", "00BF"], "Strunjan", inplace=True)
df_sea["station_id"].replace(["0DB2", "000K"], "Debeli_rtic", inplace=True)

df_sea

,datetime,station_id,Chl-a,salinity,T,SECCHI,NH4-N,NO2-N,NO3-N,PO4-P,SiO3-Si,O2,pH
0,1994-01-17 01:00:00,Strunjan,2.36,36.980000,10.62,6.5,1.51,1.390,3.15,0.03,2.43,304.130005,8.13
1,1994-01-17 01:00:00,Debeli_rtic,1.63,36.910000,10.81,4.0,2.11,2.000,3.03,0.16,3.88,247.860001,8.09
2,1994-01-17 01:00:00,Seca,1.50,36.780000,10.93,5.0,1.04,1.300,3.03,0.17,2.62,283.589996,8.12
3,1994-01-17 01:00:00,Strunjan,1.77,37.090000,10.62,NaN,1.23,1.380,2.50,0.19,2.34,284.040009,8.10
4,1994-01-17 01:00:00,Debeli_rtic,1.28,37.590000,10.92,NaN,1.76,2.570,3.70,0.26,3.70,254.119995,8.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3351,2021-12-14 10:20:00,Strunjan,NaN,37.869999,12.22,NaN,0.03,1.532,1.24,0.04,1.84,351.140015,8.19
3352,2021-12-14 12:45:00,Debeli_rtic,1.05,37.430000,12.27,5.0,0.42,1.080,4.89,0.09,5.17,352.309998,8.13
3353,2021-12-14 12:45:00,Debeli_rtic,0.71,37.660000,12.10,NaN,0.03,1.117,1.91,0.05,2.70,352.700012,8.15
3354,2021-12-14 13:25:00,Debeli_rtic,0.85,37.439999,11.68,5.0,0.03,1.173,1.95,0.03,2.58,359.329987,8.22


In [31]:
# Check for missing values
# pd.set_option("display.max_rows", 20)
df_sea.isnull().sum()

datetime         0
station_id       0
Chl-a         1002
salinity        14
T              115
SECCHI        1916
NH4-N          550
NO2-N          519
NO3-N          516
PO4-P          534
SiO3-Si        630
O2             184
pH             386
dtype: int64

### Adding chemical data to dataset

If any of these data can be obtained on a daily basis it would provide more optimal values for the preiods of interest before the biological sampling.

In [32]:
# Add appropriate sea test before timestamp of an algae sample in the consolidated dataset 3. (option to avg values of several tests inside period)
df_cons3 = df_cons2.copy()
sea_test_cols = ['Chl-a', 'salinity', 'T', 'SECCHI', 'NH4-N', 'NO2-N', 'NO3-N', 'PO4-P', 'SiO3-Si', 'O2', 'pH']
df_cons3['Chl-a'], df_cons3['salinity'], df_cons3['T'], df_cons3['SECCHI'], df_cons3['NH4-N'], df_cons3['NO2-N'], df_cons3['NO3-N'], df_cons3['PO4-P'], df_cons3['SiO3-Si'], df_cons3['O2'], df_cons3['pH'] = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,np.nan,np.nan,np.nan]

max_range = pd.Timedelta(30, unit="day")
min_range = pd.Timedelta(0, unit="day")

# Iterate over consolidated df 3 for each location seperately and select a suitable sea test (location, period).
# Iterate over the lovations of interest
for location in df_sea.station_id.unique():
    matches = 0
    for id_alg, date_alg in df_cons3.loc[df_cons3["sampling station"] == location, "date"].iteritems():
        # Iterate over ARSO df and selecet and instances in period and sum radiation values
        for id_sea, date_sea in df_sea.loc[df_sea["station_id"] == location, "datetime"].iteritems():
            # Calculate the timespan between algae sample and sea sample
            timespan = date_alg - date_sea
            #If timespan within relevant period
            if min_range <= timespan <= max_range:
                # Sea test on first acceptable sea test day (date_sea) 
                sea_test_val = df_sea.loc[id_sea, sea_test_cols]
                df_cons3.loc[id_alg, sea_test_cols] = sea_test_val
                matches += 1
                break
    print(f"Found matching test dates for {location}: {matches}")                    


Found matching test dates for Strunjan: 482
Found matching test dates for Debeli_rtic: 298
Found matching test dates for Seca: 422


In [33]:
df_cons3.isnull().sum()

date                       0
sampling station           0
sampling depth             0
sampling method            0
PSP                        5
DSP                        1
DSP_like                  60
ASP                      344
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
lipophylic_toxins        356
sun [h]                    0
air temp                   0
wind strength              0
precipitation              0
Chl-a                    642
salinity                 185
T                        239
SECCHI                   676
NH4-N                    536
NO2-N                    526
NO3-N                    522
PO4-P                    528
SiO3-Si                  593
O2                       279
pH                       410
dtype: int64

## 3.4 River inflow dataset

In [34]:
import openpyxl

# Create Soca dataframe
df_river = pd.read_excel("data/ORIGINAL2/Pretok Soča_dnevni_1994-2020.xlsx", usecols=["Datum", "SOCA (m3/s)"])  
df_river["Datum"] = pd.to_datetime(df_river["Datum"], dayfirst=True)
# # df_soca = df_soca.set_index("Year_month")
df_river = df_river.sort_values('Datum')
# df_soca = df_soca.to_period()
# df_soca["Day_of_the_Year"] = df_soca["Year_month"].dt.dayofyear

df_river

,Datum,SOCA (m3/s)
0,1994-01-01,253.608
1,1994-01-02,292.310
2,1994-01-03,167.759
3,1994-01-04,187.153
4,1994-01-05,709.643
...,...,...
9881,2020-12-27,164.943
9882,2020-12-28,158.826
9883,2020-12-29,328.918
9884,2020-12-30,413.896


In [35]:
# Create Po dataframe
df_po = pd.read_excel("data/ORIGINAL/okoljski/Pretok_Pad_1985-2020.xlsx", usecols=["Year_month", "Po"])  
df_po["Year_month"] = pd.to_datetime(df_po["Year_month"], format="%Y%m")
df_po = df_po.set_index("Year_month")
df_po = df_po.to_period()
df_po = df_po.sort_values('Year_month')
# df_po["Year"] = df_po["Year_month"].dt.year

### Adding river inflow data

In [36]:
# Loop to add appropriate summed sun river inflow before timestamp of an algae sample in the consolidated dataset.
df_cons4 = df_cons3.copy()
df_cons4["Soca"] = np.nan

max_range = pd.Timedelta(30, unit="day")
min_range = pd.Timedelta(0, unit="day")

# Iterate over consolidated df and select a date of the algae sample
for id_alg, date_alg in df_cons4["date"].iteritems():
    # Iterate over ARSO df and selecet and instances in period and sum values
    river_val = []
    for id_river, date_river in df_river["Datum"].iteritems():
        # Calculate the timespan between algae sample and river flow meassurement
        timespan = date_alg - date_river
        # If timespan within period
        if min_range <= timespan <= max_range:
            # Append value to sun_values list
            river_val.append(df_river.loc[id_river, "SOCA (m3/s)"])
        # Add sum of river flow values to df_cons once all period values are found

    df_cons4.loc[id_alg, "Soca"] = sum(river_val)

In [37]:
# Move target feature to the end
pd.set_option("display.max_rows", 20)
df_temp = df_cons4.pop("lipophylic_toxins")
df_cons4["lipophylic_toxins"] = df_temp

In [38]:
# Change date to month only
df_cons4["date"] = df_cons4["date"].dt.month

#drop irrelevant featutres
df_cons4 = df_cons4.drop(columns=["sampling station", "sampling depth", "sampling method"])
df_cons4

,date,PSP,DSP,DSP_like,ASP,Dinophysis caudata,Dinophysis fortii,Phalacroma rotundatum,Dinophysis sacculus,Dinophysis tripos,sun [h],air temp,wind strength,precipitation,Chl-a,salinity,T,SECCHI,NH4-N,NO2-N,NO3-N,PO4-P,SiO3-Si,O2,pH,Soca,lipophylic_toxins
0,5,1206.0,68.0,NaN,NaN,27.0,0.0,21.0,0.0,0.0,197.1,15.114286,2.933333,26.2,1.15,33.780000,13.580000,7.0,1.25,0.06,6.73,0.15,4.91,230.449997,8.00,3030.486,NaN
3,5,4188.0,17.0,NaN,NaN,8.0,0.0,0.0,0.0,0.0,197.1,15.114286,2.933333,26.2,0.45,36.180000,12.620000,8.0,0.60,0.04,1.12,0.12,3.02,250.100006,8.15,3030.486,NaN
6,6,0.0,27.0,NaN,NaN,16.0,3.0,5.0,0.0,0.0,166.3,18.790476,3.157143,18.8,1.11,35.530000,17.030000,NaN,1.25,0.14,1.23,0.22,1.82,274.209991,8.15,3931.331,NaN
7,6,324.0,23.0,NaN,NaN,3.0,0.0,0.0,0.0,0.0,179.1,18.533333,3.052381,23.9,1.05,35.010000,19.890000,NaN,1.28,0.12,2.52,0.09,0.83,275.109985,8.22,4090.322,NaN
8,6,0.0,20.0,NaN,NaN,8.0,0.0,0.0,0.0,0.0,157.5,19.095238,3.066667,29.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3221.804,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578,11,0.0,0.0,0.0,10800.0,0.0,0.0,0.0,0.0,0.0,118.6,8.947619,3.147619,41.9,NaN,37.660000,16.850000,6.0,NaN,NaN,NaN,NaN,NaN,295.980011,8.17,2243.638,NaN
1579,11,10.0,10.0,0.0,5000.0,0.0,0.0,10.0,0.0,0.0,118.6,8.947619,3.147619,41.9,0.80,36.889999,16.709999,5.0,0.38,0.26,1.26,0.05,2.73,307.869995,8.18,2243.638,NaN
1581,12,0.0,20.0,0.0,1800.0,0.0,0.0,0.0,0.0,0.0,64.3,7.114286,3.290476,198.0,NaN,37.660000,16.850000,6.0,NaN,NaN,NaN,NaN,NaN,295.980011,8.17,4678.399,NaN
1580,12,10.0,20.0,0.0,1200.0,0.0,0.0,10.0,0.0,0.0,64.3,7.114286,3.290476,198.0,0.76,37.650002,16.510000,6.0,0.17,0.23,0.61,0.04,1.70,308.959991,8.16,4678.399,NaN


In [39]:
pd.set_option('display.max_rows', 30)
df_cons4.isnull().sum()

date                       0
PSP                        5
DSP                        1
DSP_like                  60
ASP                      344
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
sun [h]                    0
air temp                   0
wind strength              0
precipitation              0
Chl-a                    642
salinity                 185
T                        239
SECCHI                   676
NH4-N                    536
NO2-N                    526
NO3-N                    522
PO4-P                    528
SiO3-Si                  593
O2                       279
pH                       410
Soca                       0
lipophylic_toxins        356
dtype: int64

In [40]:
df_cons4["lipophylic_toxins"].value_counts(dropna=False)

neg    901
NaN    356
poz    126
Name: lipophylic_toxins, dtype: int64

In [41]:
# Create pickle
df_cons4.to_pickle(f"objects/df_cons4-{nb_fname}")

In [42]:
# Export csv
df_cons.to_csv(f"data/Weka/cons_{nb_fname}.csv", sep=",", na_rep="?", index=False)

In [43]:
# C5 export function 
def C5_export(df, file_prefix, target, max_values=10):
    with open(file_prefix + ".names", "w") as fp:
        print("{}.\n".format(target), file=fp)
        columns = df.columns
        for col in columns:
            values = [str(x) for x in set(df[col].value_counts().index) if x !="?"]
            if len(values) > max_values:
                print("{}: continuous.".format(col), file=fp)
            else:
                print("{}: {}.".format(col, ",".join(values)), file=fp)
    df.to_csv(file_prefix + ".data", header=False, na_rep="?", index=False)

# target = "lipophylic_toxins"
# C5_export(df_cons4, nb_fname, target, max_values=12)